In [2]:
import pandas as pd
import csv
import numpy as np
import os

# Dictionaries

In [4]:
# Change the values that need to be changed manually here
station_coord_rects = {
    'Xavier Hall': { 
        'Latitude': { 
            'lower': 14.640004,
            'higher': 14.640371
        },
        'Longitude': { 
            'lower': 121.078251,
            'higher': 121.078789
        }
    },
    'Fine Arts Annex': { 
        'Latitude': { 
            'lower': 14.636606,
            'higher': 14.636749
        },
        'Longitude': { 
            'lower': 121.078116,
            'higher': 121.078324
        }
    },
    'Loyola House of Studies': { 
        'Latitude': { 
            'lower': 14.636115,
            'higher': 14.636302
        },
        'Longitude': { 
            'lower': 121.080644,
            'higher': 121.080738
        }
    },
    'Grade School': { 
        'Latitude': { 
            'lower': 14.634715,
            'higher': 14.634715
        },
        'Longitude': { 
            'lower': 121.076171,
            'higher': 121.076326
        }
    },
    'JSEC': { 
        'Latitude': { 
            'lower': 14.637686,
            'higher': 14.637804
        },
        'Longitude': { 
            'lower': 121.076260,
            'higher': 121.076370
        }
    },
    'Gate 2.5': { 
        'Latitude': { 
            'lower': 14.637874,
            'higher': 14.637927
        },
        'Longitude': { 
            'lower': 121.074848,
            'higher': 121.075020
        }
    },
    'Leong Hall': { 
        'Latitude': { 
            'lower': 14.640724,
            'higher': 14.640864
        },
        'Longitude': { 
            'lower': 121.076177,
            'higher': 121.076395
        }
    },
    'Power Station': { 
        'Latitude': { 
            'lower': 14.635504,
            'higher': 14.635692
        },
        'Longitude': { 
            'lower': 121.074559,
            'higher': 121.074674
        }
    },
}

In [5]:
next_station_dict = { 
    'Xavier Hall': 'Fine Arts Annex',
    'Fine Arts Annex': 'Loyola House of Studies',
    'Loyola House of Studies': 'Grade School',
    'Grade School': 'Gate 2.5',
    'Gate 2.5': 'Leong Hall',
    'Leong Hall': 'Xavier Hall'
}

# Functions

In [6]:
from datetime import time

def time_to_daytimestamp(stamp: str) -> int:
    if stamp is None:
        return -1
    
    if isinstance(stamp, float):
        return -1
    
    if isinstance(stamp, time):
        stamp = stamp.strftime("%H:%M:%S")
    
    h, m, s = stamp.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [7]:
# Reformat Lat and Long to be decimal coordinates
def ddmm_mmmm_to_decimal(coord):
    # Assuming coord is in the format 'ddmm.mmmm'
    
    # Extract degrees and decimal minutes
    degrees = coord // 100
    decimal_minutes = coord % 100
    
    # Convert decimal minutes to decimal degrees
    decimal_degrees = degrees + decimal_minutes / 60
    
    return decimal_degrees

In [8]:
# Label stations along the line that the ejeep is in based on coordinates
def coord_to_station(lat, long) -> str:
    stations = station_coord_rects.keys()

    for station in stations:
        lat_bounds, long_bounds = station_coord_rects[station]['Latitude'], station_coord_rects[station]['Longitude']

        if (lat >= lat_bounds['lower'] - 0.0003 and lat <= lat_bounds['higher'] + 0.0003) and (long >= long_bounds['lower'] - 0.0003 and long <= long_bounds['higher'] + 0.0003):
            return station
    else:
        return "En Route"

In [9]:
import math

# Function to calculate distance using Haversine formula
def haversine_distance(lat_diff, lon_diff):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude differences from degrees to radians
    lat_diff_rad = math.radians(lat_diff)
    lon_diff_rad = math.radians(lon_diff)

    # Haversine formula
    a = math.sin(lat_diff_rad / 2)**2 + math.cos(0) * math.cos(0) * math.sin(lon_diff_rad / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate distance
    distance = R * c

    return distance * 1000

In [10]:
# Make dictionary of all stations & average lat & long
def station_centroids(coord) -> dict:
    new_dict = dict()

    new_dict['En Route'] = None

    for station in station_coord_rects:
        ave = (station_coord_rects[station][coord]['lower'] + station_coord_rects[station][coord]['higher']) / 2

        new_dict[station] = ave
    
    return new_dict

lat_centers = station_centroids('Latitude')
long_centers = station_centroids('Longitude')

lat_centers
long_centers

{'En Route': None,
 'Xavier Hall': 121.07852,
 'Fine Arts Annex': 121.07821999999999,
 'Loyola House of Studies': 121.080691,
 'Grade School': 121.07624849999999,
 'JSEC': 121.076315,
 'Gate 2.5': 121.074934,
 'Leong Hall': 121.07628600000001,
 'Power Station': 121.07461649999999}

In [11]:
# Define Prev Station
def prev_station(df):
    df['Prev Station'] = df['Station']
    df.loc[df['Station'] == 'En Route', "Prev Station"] = np.nan

    df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
    df.loc[df['Station'] != 'En Route', "Prev Station"] = np.nan
    df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
    df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
    
    return df

In [12]:
# Define Next Station
def next_station(df):
    df['Next Station'] = df['Station']
    df.loc[df['Station'] == 'En Route', "Next Station"] = np.nan

    df['Next Station'] = df['Next Station'].fillna(method='bfill')
    df.loc[df['Station'] != 'En Route', "Next Station"] = np.nan
    df['Next Station'] = df['Next Station'].fillna(method='bfill')
    df['Next Station'] = df['Next Station'].fillna(method='ffill')
    
    return df

In [13]:
# Previous & Next Times

def prev_next_times(df):
    # Create a 2nd df that's a more compiled version of the table for easier look-ups

    df2 = pd.DataFrame(columns=['Original Index', 'Station', 'Time'])

    next_station = ''

    for index, row in df.iterrows():
        if row['Station'] == 'En Route' or row['Station'] == 'Power Station':
            continue

        # Log the first instance of the ejeep getting to the stop
        if row['Station'] == next_station or next_station == '':
            new_row = {
                'Original Index': index,
                'Station': row['Station'],
                'Time': row['Time']
            }
            df2.loc[len(df2)] = new_row

            next_station = next_station_dict[row['Station']]
            
    # This essentially notes when the e-jeep arrives and leaves at the station

    df_stations_sum = pd.DataFrame(columns=['Original Index', 'Station', 'Time', 'Status'])

    df_stations_sum = pd.DataFrame(columns=['Original Index', 'Station', 'Time', 'Status'])

    previous_index = None
    previous_station = 'En Route'
    previous_time = None

    for index, row in df.iterrows():
        current_time, current_station = row['Time'], row['Station']

        if previous_station == 'En Route' and current_station == 'En Route':
            continue

        if previous_station == 'En Route' and current_station != 'En Route':
            # Arrived at the station
            new_row = { 
                'Original Index': index,
                'Station': current_station,
                'Time': current_time,
                'Status': 'Arriving'
            }

            df_stations_sum.loc[len(df_stations_sum)] = new_row
        
        if previous_station != 'En Route' and current_station == 'En Route':
            # Departed from the station
            new_row = { 
                'Original Index': previous_index,
                'Station': previous_station,
                'Time': previous_time,
                'Status': 'Departing'
            }

            df_stations_sum.loc[len(df_stations_sum)] = new_row
        
        previous_time, previous_index, previous_station = current_time, index, current_station
    
    df['Previous Time to Xavier Hall'] = None
    df['Previous Time to Fine Arts Annex'] = None
    df['Previous Time to Loyola House of Studies'] = None
    df['Previous Time to Grade School'] = None
    df['Previous Time to Gate 2.5'] = None
    df['Previous Time to Leong Hall'] = None

    df['Next Time to Xavier Hall'] = None
    df['Next Time to Fine Arts Annex'] = None
    df['Next Time to Loyola House of Studies'] = None
    df['Next Time to Grade School'] = None
    df['Next Time to Gate 2.5'] = None
    df['Next Time to Leong Hall'] = None
    
    df2 = df_stations_sum.copy()

    for index, row in df2.iterrows():
        future_df = df2.iloc[index + 1: index + 40]
        prev_df = df2.iloc[max(index - 40, 0): max(0, index - 1)]

        if row['Status'] == 'Departing':
            for station in next_station_dict.keys():
                future_df_index = future_df.index.where(future_df['Station'] == station).min()
                if not pd.isnull(future_df_index):
                    future_df_index = int(future_df_index)
                    df.at[row['Original Index'], f'Next Time to {station}'] = df2.at[future_df_index, 'Time']
        elif row['Status'] == 'Arriving':
            for station in next_station_dict.keys():
                prev_df_index = prev_df.index.where(prev_df['Station'] == station).max()
                if not pd.isnull(prev_df_index):
                    prev_df_index = int(prev_df_index)
                    df.at[row['Original Index'], f'Previous Time to {station}'] = df2.at[prev_df_index, 'Time']
                    
    df = df.dropna(how='all')
    
    for station in next_station_dict.keys():
        df[f'Previous Time to {station}'] = df[f'Previous Time to {station}'].interpolate(method='ffill', limit_direction='forward')
        df[f'Next Time to {station}'] = df[f'Next Time to {station}'].interpolate(method='bfill', limit_direction='backward')
    
    return df

In [27]:
def process_station(file_path, jeep_num):
     df = pd.read_csv(file_path)
     df.tail(10)

     df.columns = ['Type', 'Datetime', 'Lat', 'Long', 'Altitude', 'Humidity', 'Temperature']
     df.drop(columns='Type', inplace=True)

     # Drop rows with null at start and end
     # df = df.loc[df.notnull().all(axis=1).argmax():]

     # ids = df[df.notnull().all(axis=1)].index
     # last_id = ids[-1]

     # df = df.loc[:last_id]
            
     # Fix time to PHT
     log_date = os.path.basename(file_path).split('/')[-1]
     df['Datetime'] = pd.to_datetime(df['Datetime'], format='mixed')
     df['Datetime'] = df['Datetime'].dt.tz_convert('+08:00')
     df['Datetime'] = pd.to_datetime(df['Datetime'].dt.strftime(f"{log_date[5:15]} %H:%M:%S"))
     
     # Create a DataFrame with a time range for every second between 6am and 6pm
     time_range = pd.date_range(log_date[5:15] + " 06:00:00", log_date[5:15] + " 18:00:00", freq='S')
     expected_df = pd.DataFrame({'Datetime': time_range})
     expected_df['Datetime'] = pd.to_datetime(expected_df['Datetime'])

     # Merge the expected DataFrame with your original DataFrame on the 'time' column
     df = pd.merge(expected_df, df, on='Datetime', how='left')
     
     # Extract Time
     df['Time'] = df['Datetime'].dt.time

     # Convert Time to Seconds from Midnight
     df['Time'] = df['Time'].apply(lambda val: time_to_daytimestamp(val))
     
     # Extract Day
     df['Day'] = df['Datetime'].dt.day

     # Add day of week
     # 0 = Monday, 6 = Sunday
     df['Day_of_Week'] = df['Datetime'].dt.day_of_week
            
     # Add hour of day
     df['Hour_of_Day'] = df['Datetime'].dt.hour
     
     # Filter out rows before 6am and after 6pm
     df = df.loc[(df['Hour_of_Day'] >= 6) & (df['Hour_oDay'] <= 18)]

     # Get One Hot Encoded E-Jeep ID
     rpi_id = file_path.split('/')[1]
     df['IsEJeep1'] = 0
     df['IsEJeep2'] = 0
     df['IsEJeep3'] = 0
     df[f"IsEJeep{jeep_num}"] = 1

     # Iterpolate values for NaN GPS values
     df['Lat'] = df['Lat'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Long'] = df['Long'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Altitude'] = df['Altitude'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Temperature'] = df['Temperature'].astype(float).interpolate(method='linear', limit_direction='both') 
     df['Humidity'] = df['Humidity'].astype(float).interpolate(method='linear', limit_direction='both') 
            
     df['Latitude'] = df.apply(lambda row: ddmm_mmmm_to_decimal(row['Lat']), axis=1)
     df['Longitude'] = df.apply(lambda row: ddmm_mmmm_to_decimal(row['Long']), axis=1)

     # Drop old Lat / Long values
     df.drop(columns=['Lat', 'Long'], inplace=True)
            
     df['Station'] = df.apply(lambda row: coord_to_station(row['Latitude'], row['Longitude']), axis=1)
            
     # Calculate speed (noted as distance since it's by second anyway) based on change in coordinates from the previous
     # Helps establish if ejeep is truly within station

     df['Lat Diff'] = df['Latitude'].diff()
     df['Long Diff'] = df['Longitude'].diff()

     df['Distance'] = df.apply(lambda row: haversine_distance(row['Lat Diff'], row['Long Diff']), axis=1)
     
     df['Lat Diff'] = df['Lat Diff'].fillna(0)
     df['Long Diff'] = df['Long Diff'].fillna(0)
     df['Distance'] = df['Distance'].fillna(0)
     
     df['Station'] = df.apply(lambda row: 'En Route' if row['Station'] != 'En Route' and row['Distance'] >= 0.5 else row['Station'], axis=1)

     # Mark if it's in the Charging Station
     df['IsCharging'] = 0
     df.loc[df['Station'] == 'Power Station', "IsCharging"] = 1
            
     # Add station column that's true if in station, false if en route
     df["IsStation"] = 0
     df.loc[(df["Station"] != "En Route") & (df["Station"] != "Power Station"), "IsStation"] = 1
     
     # Drop rows before first and last station of each day
     # id_not_null = df.notnull().all(axis=1).idxmax()
     # start = df.loc[df.IsStation==1,'IsStation'].index[0] - id_not_null
     # end = df.query('IsStation == 1').index.max()+1
     # df = df.iloc[start:end]
     
     df = prev_station(df)
     df = next_station(df)
            
     # Filter out repeated stations
     df['Station'] = df.apply(lambda x: x['Next Station'] if x['Station'] == 'En Route' and x['Prev Station'] == x['Next Station'] else x['Station'], axis=1)
     df = prev_station(df)
     df = next_station(df)
            
     df = prev_next_times(df)

     df['Cum Distance'] = df[::-1].groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
     df.loc[df['Station'] != 'En Route', 'Cum Distance'] = 0

     # Add distances (as the crow flies) to next station
     df['Next Station Lat'] = df['Next Station'].map(lat_centers)
     df['Next Station Long'] = df['Next Station'].map(long_centers)

     df['Abs Distance'] = df.apply(lambda row: haversine_distance(row['Next Station Lat'] - row['Latitude'], row['Next Station Long'] - row['Longitude']), axis=1)

     # Do percentage of distance remaining to next station

     # Calculate the maximum value of C within each group of 'X'
     max_distance_per_group = df[df['Station'] == 'En Route'].groupby((df['Station'] != 'En Route').cumsum())['Cum Distance'].transform('max')

     # Add column D representing the percentage of each value in column C relative to the maximum value within its group
     df['Percent Distance'] = df['Cum Distance'] / max_distance_per_group * 100

     # Add distance (as the crow flies) to the previous station
     df['Prev Station Lat'] = df['Prev Station'].map(lat_centers)
     df['Prev Station Long'] = df['Prev Station'].map(long_centers)

     df['Prev Abs Distance'] = df.apply(lambda row: haversine_distance(row['Prev Station Lat'] - row['Latitude'], row['Prev Station Long'] - row['Longitude']), axis=1)

     # Add distance (by road) to the previous station
     df['Prev Cum Distance'] = df.groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
     df.loc[df['Station'] != 'En Route', 'Prev Cum Distance'] = 0

     ### Ordinal Encoding
     route = ['En Route', 'Grade School', 'JSEC', 'Gate 2.5', 'Leong Hall', 'Xavier Hall', 'University Dormitory', 'Fine Arts Annex', 'Loyola House of Studies', 'Gate 1', 'Power Station']

     # Station
     df['Encoded Station'] = df['Station'].apply(lambda x: route.index(x))

     # Prev Station
     df['Encoded Prev Station'] = df['Prev Station'].apply(lambda x: route.index(x))

     # Next Station
     df['Encoded Next Station'] = df['Next Station'].apply(lambda x: route.index(x))

     df.drop(columns='Station', inplace=True)
     df.drop(columns='Prev Station', inplace=True)
     df.drop(columns='Next Station', inplace=True)

     df = df.fillna(-1)

     df = df[['Datetime', 'Day', 'Day_of_Week', 'Time', 'Hour_of_Day', 'IsEJeep1', 'IsEJeep2', 'IsEJeep3', 'Altitude', 'Humidity', 
         'Temperature', 'Latitude', 'Longitude', 'Lat Diff', 'Long Diff', 'Distance',  'Cum Distance', 'Abs Distance', 'Percent Distance', 
         'Encoded Station', 'IsStation', 'IsCharging', 
         'Encoded Prev Station', 'Prev Station Lat', 'Prev Station Long', 'Prev Abs Distance', 'Prev Cum Distance', 
         'Previous Time to Xavier Hall', 'Previous Time to Fine Arts Annex', 'Previous Time to Loyola House of Studies',
         'Previous Time to Grade School', 'Previous Time to Gate 2.5', 'Previous Time to Leong Hall', 
         'Encoded Next Station', 'Next Station Lat', 'Next Station Long', 'Next Time to Xavier Hall', 'Next Time to Fine Arts Annex', 
         'Next Time to Loyola House of Studies', 'Next Time to Grade School', 'Next Time to Gate 2.5', 'Next Time to Leong Hall', 
        ]]

     print("done with " + log_date)
     return df

In [24]:
df = process_station("../E-Jeep Data/rpi1/data_2023-11-09.csv")
df

/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3185133074.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a f

done with data_2023-11-09.csv


,Datetime,Day,Day_of_Week,Time,Hour_of_Day,IsEJeep1,IsEJeep2,IsEJeep3,Altitude,Humidity,...,Previous Time to Leong Hall,Encoded Next Station,Next Station Lat,Next Station Long,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
0,2023-11-09 06:00:00,9,3,21600,6,0,0,0,50.3,77.699997,...,-1.0,1,14.634715,121.076248,23020.0,24489.0,23214.0,22598.0,23752.0,22849.0
1,2023-11-09 06:00:01,9,3,21601,6,0,0,0,50.1,77.699997,...,-1.0,1,14.634715,121.076248,23020.0,24489.0,23214.0,22598.0,23752.0,22849.0
2,2023-11-09 06:00:02,9,3,21602,6,0,0,0,50.1,77.699997,...,-1.0,1,14.634715,121.076248,23020.0,24489.0,23214.0,22598.0,23752.0,22849.0
3,2023-11-09 06:00:03,9,3,21603,6,0,0,0,49.8,77.699997,...,-1.0,1,14.634715,121.076248,23020.0,24489.0,23214.0,22598.0,23752.0,22849.0
4,2023-11-09 06:00:04,9,3,21604,6,0,0,0,49.8,77.699997,...,-1.0,1,14.634715,121.076248,23020.0,24489.0,23214.0,22598.0,23752.0,22849.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43199,2023-11-09 17:59:56,9,3,64796,17,0,0,0,12.5,75.400002,...,63745.0,10,14.635598,121.074616,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
43200,2023-11-09 17:59:57,9,3,64797,17,0,0,0,11.5,75.400002,...,63745.0,10,14.635598,121.074616,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
43201,2023-11-09 17:59:58,9,3,64798,17,0,0,0,10.8,75.400002,...,63745.0,10,14.635598,121.074616,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
43202,2023-11-09 17:59:59,9,3,64799,17,0,0,0,10.5,75.400002,...,63745.0,10,14.635598,121.074616,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


# Raw Data
* Combine csv files from each rpi into one data frame
* Pre-process raw data

In [28]:
# Create a list to hold the dataframes
df_list = []

for i in range(1, 4):
    folder_path = os.path.join('../E-Jeep Data', f"rpi{i}")
    all_files = os.listdir(folder_path)
    
    # Filter out non-CSV files
    csv_files = [f for f in all_files if f.endswith('.csv')]
    
    for csv in csv_files:
        file_path = os.path.join(folder_path, csv)
        try:
            # Try reading the file using default UTF-8 encoding
            df = process_station(file_path, i)
            df_list.append(df)
        except UnicodeDecodeError:
            try:
                # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
                df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
                df_list.append(df)
            except Exception as e:
                print(f"Could not read file {csv} because of error: {e}")
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")

# Concatenate all data into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df.to_csv('Finalized Out.csv')

/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3185133074.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a f

done with data_2023-11-17_1.csv


/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/773614971.py:2: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev

done with data_2023-10-23.csv


/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3185133074.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a f

...

/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3185133074.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a f

Could not read file data_2023-11-13.csv because of error: nan is not in list
done with data_2023-11-20_2.csv


/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='ffill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3679190830.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Prev Station'] = df['Prev Station'].fillna(method='bfill')
/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/3185133074.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a f

Could not read file data_2023-11-14_3.csv because of error: nan is not in list


/var/folders/4b/fm5gnnzx6qqc7t0l504grbjm0000gn/T/ipykernel_28892/543211647.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list, ignore_index=True)


In [33]:
# df.drop(columns='Datetime', inplace=True)
df.to_csv('Finalized Out.csv')

In [ ]:
# Sort by datetime
df = df.sort_values(by=['Datetime'], ascending=True)

# Data Processing

In [ ]:
# Add IsCharging column: true if charging, false otherwise
df['IsCharging'] = 0

df.loc[df['Station'] == 'Power Station', "IsCharging"] = 1

In [ ]:
# Add distances (by road) to next station
df['Cum Distance'] = df[::-1].groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
df.loc[df['Station'] != 'En Route', 'Cum Distance'] = 0

df.dtypes


Datetime                                    datetime64[ns]
Altitude                                           float64
Humidity                                           float64
Temperature                                        float64
Time                                                object
Day                                                float64
Day_of_Week                                         object
Hour_of_Day                                        float64
IsEJeep1                                             int64
IsEJeep2                                             int64
IsEJeep3                                             int64
Latitude                                           float64
Longitude                                          float64
Station                                             object
Lat Diff                                           float64
Long Diff                                          float64
Distance                                           float

In [ ]:
# Add distances (as the bird flies) to next station
df['Next Station Lat'] = df['Next Station'].map(lat_centers)
df['Next Station Long'] = df['Next Station'].map(long_centers)

df['Abs Distance'] = df.apply(lambda row: haversine_distance(row['Next Station Lat'] - row['Latitude'], row['Next Station Long'] - row['Longitude']), axis=1)

In [ ]:
# Do percentage of distance remaining to next station

# Calculate the maximum value of C within each group of 'X'
max_distance_per_group = df[df['Station'] == 'En Route'].groupby((df['Station'] != 'En Route').cumsum())['Cum Distance'].transform('max')

# Add column D representing the percentage of each value in column C relative to the maximum value within its group
df['Percent Distance'] = df['Cum Distance'] / max_distance_per_group * 100

df.dtypes


Datetime                                    datetime64[ns]
Altitude                                           float64
Humidity                                           float64
Temperature                                        float64
Time                                                object
Day                                                float64
Day_of_Week                                         object
Hour_of_Day                                        float64
IsEJeep1                                             int64
IsEJeep2                                             int64
IsEJeep3                                             int64
Latitude                                           float64
Longitude                                          float64
Station                                             object
Lat Diff                                           float64
Long Diff                                          float64
Distance                                           float

In [ ]:
# Add distance (as the crow flies) to the previous station
df['Prev Station Lat'] = df['Prev Station'].map(lat_centers)
df['Prev Station Long'] = df['Prev Station'].map(long_centers)

df['Prev Abs Distance'] = df.apply(lambda row: haversine_distance(row['Prev Station Lat'] - row['Latitude'], row['Prev Station Long'] - row['Longitude']), axis=1)

In [ ]:
# Add distance (by road) to the previous station
df['Prev Cum Distance'] = df.groupby((df['Station'] != 'En Route').cumsum())['Distance'].cumsum()
df.loc[df['Station'] != 'En Route', 'Prev Cum Distance'] = 0

In [ ]:
# Global variables


# Average speed every 10 seconds

# Average, min, max temp & humidity in between stations



# Ordinal Encoding

In [ ]:
# Day of Week
df.sort_values('Day_of_Week')
df['Encoded Day'] = pd.factorize(df['Day_of_Week'])[0] + 1

In [ ]:
route = ['En Route', 'Grade School', 'JSEC', 'Gate 2.5', 'Leong Hall', 'Xavier Hall', 'University Dormitory', 'Fine Arts Annex', 'Loyola House of Studies', 'Gate 1', 'Power Station']

# Station
df['Encoded Station'] = df['Station'].apply(lambda x: route.index(x))

# Prev Station
df['Encoded Prev Station'] = df['Prev Station'].apply(lambda x: route.index(x))

# Next Station
df['Encoded Next Station'] = df['Next Station'].apply(lambda x: route.index(x))

In [ ]:
df.drop(columns='Day_of_Week', inplace=True)
df.drop(columns='Station', inplace=True)
df.drop(columns='Prev Station', inplace=True)
df.drop(columns='Next Station', inplace=True)

In [ ]:
# make sure there are no nan values
df = df.fillna(-1)

In [ ]:
# Rearrange columns

df = df[['Datetime', 'Day', 'Encoded Day', 'Time', 'Hour_of_Day', 'IsEJeep1', 'IsEJeep2', 'IsEJeep3', 'Altitude', 'Humidity', 
         'Temperature', 'Latitude', 'Longitude', 'Lat Diff', 'Long Diff', 'Distance',  'Cum Distance', 'Abs Distance', 'Percent Distance', 
         'Encoded Station', 'IsStation', 'IsCharging', 
         'Encoded Prev Station', 'Prev Station Lat', 'Prev Station Long', 'Prev Abs Distance', 'Prev Cum Distance', 
         'Previous Time to Xavier Hall', 'Previous Time to Fine Arts Annex', 'Previous Time to Loyola House of Studies',
         'Previous Time to Grade School', 'Previous Time to Gate 2.5', 'Previous Time to Leong Hall', 
         'Encoded Next Station', 'Next Station Lat', 'Next Station Long', 'Next Time to Xavier Hall', 'Next Time to Fine Arts Annex', 
         'Next Time to Loyola House of Studies', 'Next Time to Grade School', 'Next Time to Gate 2.5', 'Next Time to Leong Hall', 
        ]]

df

,Datetime,Day,Encoded Day,Time,Hour_of_Day,IsEJeep1,IsEJeep2,IsEJeep3,Altitude,Humidity,...,Previous Time to Leong Hall,Encoded Next Station,Next Station Lat,Next Station Long,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall
245434,2023-11-09 06:16:38,9.0,1,06:16:38,6.0,1,0,0,57.8,78.400002,...,-1,4,14.640794,121.076286,06:20:49,06:43:01,06:23:40,06:26:54,06:33:19,06:35:52
245435,2023-11-09 06:16:39,9.0,1,06:16:39,6.0,1,0,0,58.5,78.400002,...,-1,4,14.640794,121.076286,06:40:22,06:43:01,06:23:40,06:26:54,06:33:19,06:35:52
245436,2023-11-09 06:16:40,9.0,1,06:16:40,6.0,1,0,0,57.9,78.199997,...,-1,4,14.640794,121.076286,06:40:22,06:43:01,06:23:40,06:26:54,06:33:19,06:35:52
245437,2023-11-09 06:16:41,9.0,1,06:16:41,6.0,1,0,0,57.7,78.199997,...,-1,4,14.640794,121.076286,06:40:22,06:43:01,06:23:40,06:26:54,06:33:19,06:35:52
245438,2023-11-09 06:16:42,9.0,1,06:16:42,6.0,1,0,0,57.2,78.199997,...,-1,4,14.640794,121.076286,06:40:22,06:43:01,06:23:40,06:26:54,06:33:19,06:35:52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42884,2023-11-17 18:01:11,17.0,2,18:01:11,18.0,1,0,0,52.8,49.400002,...,17:44:42,10,14.635598,121.074616,-1,-1,-1,-1,-1,-1
42885,2023-11-17 18:01:12,17.0,2,18:01:12,18.0,1,0,0,53.7,49.400002,...,17:44:42,10,14.635598,121.074616,-1,-1,-1,-1,-1,-1
42886,2023-11-17 18:01:13,17.0,2,18:01:13,18.0,1,0,0,54.4,49.400002,...,17:44:42,10,14.635598,121.074616,-1,-1,-1,-1,-1,-1
42887,2023-11-17 18:01:14,17.0,2,18:01:14,18.0,1,0,0,55.1,49.400002,...,17:44:42,10,14.635598,121.074616,-1,-1,-1,-1,-1,-1


# Convert Datetime to Numeric Values

In [ ]:
# Convert all the times into a timestamp with 0 = 00:00:00 and 86399 = 23:59:59
from datetime import datetime

def time_to_daytimestamp(stamp: str) -> int:
    if stamp is None:
        return -1
    
    if isinstance(stamp, float):
        return -1
    
    h, m, s = stamp.split(':')
    return int(h) * 60 * 60 + int(m) * 60 + int(s)

def datetime_to_daytimestamp(stamp: str) -> int:
    if stamp is None:
        return -1
    
    if isinstance(stamp, float):
        return -1
    
    # Parse the datetime string into a datetime object
    dt_object = datetime.strptime(stamp, "%Y-%m-%d %H:%M:%S")

    # Calculate the total minutes from midnight
    return dt_object.hour * 60 + dt_object.minute

df['Datetime'] = df['Datetime'].apply(lambda time: datetime_to_daytimestamp(time))
df['Time'] = df['Time'].apply(lambda time: time_to_daytimestamp(time))

df['Previous Time to Xavier Hall'] = df['Previous Time to Xavier Hall'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Fine Arts Annex'] = df['Previous Time to Fine Arts Annex'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Loyola House of Studies'] = df['Previous Time to Loyola House of Studies'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Grade School'] = df['Previous Time to Grade School'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Gate 2.5'] = df['Previous Time to Gate 2.5'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Leong Hall'] = df['Previous Time to Leong Hall'].apply(lambda time: time_to_daytimestamp(time))

df['Next Time to Xavier Hall'] = df['Next Time to Xavier Hall'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Fine Arts Annex'] = df['Next Time to Fine Arts Annex'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Loyola House of Studies'] = df['Next Time to Loyola House of Studies'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Grade School'] = df['Next Time to Grade School'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Gate 2.5'] = df['Next Time to Gate 2.5'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Leong Hall'] = df['Next Time to Leong Hall'].apply(lambda time: time_to_daytimestamp(time))

df

TypeError: strptime() argument 1 must be str, not Timestamp

In [ ]:
df.to_csv('Out.csv')